In [3]:
import cv2
import mediapipe as mp
import time
import math
import numpy as np
import pycaw

class handDetector():
    def _init_(self, mode=False, maxHands=2, detectionCon=0.5, trackCon=0.5):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionCon = detectionCon
        self.trackCon = trackCon

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(self.mode, self.maxHands, self.detectionCon, self.trackCon)
        self.mpDraw = mp.solutions.drawing_utils #для отрисования линий между точка Lm

    def findhands(self, img, draw = True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.res = self.hands.process(imgRGB)

        if self.res.multi_hand_landmarks:
            for handLm in self.res.multi_hand_landmarks:
#                for id, lm, in enumerate(handLm.landmark):
#                   h,w,c = img.shape
#                    cx,cy = int(lm.x*w), int(lm.y*h)
#                    if id == 0:
#                        cv2.circle(img, (cx,cy), 12, (255,0,255), cv2.FILLED)
                if draw:
                    self.mpDraw.draw_landmarks(img, handLm, self.mpHands.HAND_CONNECTIONS)
        return img
    
    def findPosition(self, img, handNum = 0, draw = True):
        
        Lmlst = []
        if self.res.multi_hand_landmarks:
            MyHand = self. res.multi_hand_landmarks[handNum]
            for id, lm, in enumerate(MyHand.landmark):
                h,w,c = img.shape
                cx,cy = int(lm.x*w), int(lm.y*h)
                Lmlst.append([id,cx,cy])
                #if draw:
                #    cv2.circle(img, (cx,cy), 12, (255,0,255), cv2.FILLED)
        return Lmlst

In [5]:
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = cast(interface, POINTER(IAudioEndpointVolume))
#volume.GetMute()
#volume.GetMasterVolumeLevel()
volRange = volume.GetVolumeRange()
volume.SetMasterVolumeLevel(0.0, None)
minVol = volRange[0]
maxVol = volRange[1]

ImportError: cannot import name 'COMError' from '_ctypes' (/opt/anaconda3/lib/python3.9/lib-dynload/_ctypes.cpython-39-darwin.so)

In [ ]:
сap = cv2.VideoCapture(0)
codec = cv2.VideoWriter_fourcc(*'XVID')
output = cv2.VideoWriter('CAPTURE.avi', codec, 30, (640, 480))
recording_flag = False
pTime = 0
cTime = 0
detector = handDetector()
detector._init_()
vol = 0
vol2 = 300
maxx = 0

while True:
    res, img = сap.read()
    img = detector.findhands(img)
    lst = detector.findPosition(img)
    
    if len(lst) != 0:
        x1, y1 = lst[4][1], lst[4][2]
        x2, y2 = lst[8][1], lst[8][2]
        cx, cy = (x1 + x2)//2, (y1 + y2)//2
        cv2.line(img, (x1,y1), (x2,y2), (255,0,255),1)
        cv2.circle(img, (cx,cy), 5, (123,123,0), cv2.FILLED)
        
        length1 = math.hypot(x2-x1,y2-y1)
        
        if (length1) < 20:
            cv2.circle(img, (cx,cy), 5, (123,0,0), cv2.FILLED)
            
        x3, y3 = lst[16][1], lst[16][2]
        x4, y4 = lst[12][1], lst[12][2]
        length2 = math.hypot(x4-x3,y4-y3)
        
        cv2.putText(img,"Vol control:",(15,320), cv2.FONT_HERSHEY_PLAIN,1,
        (0,0,0),2)
        
        if length2 > 20:
            vol = np.interp(length1,[20,115],[maxVol, minVol])
            volume.SetMasterVolumeLevel(vol, None)
        
            vol2 = np.interp(length1,[20,115],[150, 300])
            cv2.putText(img,"on",(55,340), cv2.FONT_HERSHEY_PLAIN,1,
            (0,0,255),2)
        else:
            cv2.putText(img,"off",(55,340), cv2.FONT_HERSHEY_PLAIN,1,
            (0,0,255),2)
            
            
        cv2.rectangle(img, (50,150), (75,300), (0,0,0),3)
        cv2.rectangle(img, (50,int(vol2)), (75,300), (0,255,0), cv2.FILLED)
        
        
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img,str(int(fps)),(10,70), cv2.FONT_HERSHEY_PLAIN,3,
    (255,0,255),3) #(куда, что, позиция, фонт какой-то, размер, цвет, толщина)

    cv2.imshow('From Camera', img)
    key = cv2.waitKey(1)
        
         
    if key%256 == 27:
        print(maxx)
        break
    elif key%256 == 32:
        if recording_flag == False:
            output = cv2.VideoWriter('CAPTURE.avi', codec, 30, (640, 480))
            recording_flag = True
        else:
            recording_flag = False

    if recording_flag:
        output.write(img)

cap.release()
output.release()
cv2.destroyAllWindows()
